In [6]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
# pd.set_option('future.no_silent_downcasting', True)

In [7]:
from datetime import datetime, timedelta

def generate_month_year_list(start_year, start_month, n_months=36):
    start_month = datetime(year=start_year, month=start_month, day=1)
    month_year_list = []

    for i in range(n_months):
        # Format the month and year as "Month Year"
        month_year = start_month.strftime("%B %Y")
        month_year_list.append(month_year)
        
        # Move to the previous month
        # Use timedelta to avoid issues with months of varying lengths
        first_day_of_current_month = start_month.replace(day=1)
        start_month = first_day_of_current_month - timedelta(days=1)
    
    return month_year_list

generate_month_year_list(2024, 8)

['August 2024',
 'July 2024',
 'June 2024',
 'May 2024',
 'April 2024',
 'March 2024',
 'February 2024',
 'January 2024',
 'December 2023',
 'November 2023',
 'October 2023',
 'September 2023',
 'August 2023',
 'July 2023',
 'June 2023',
 'May 2023',
 'April 2023',
 'March 2023',
 'February 2023',
 'January 2023',
 'December 2022',
 'November 2022',
 'October 2022',
 'September 2022',
 'August 2022',
 'July 2022',
 'June 2022',
 'May 2022',
 'April 2022',
 'March 2022',
 'February 2022',
 'January 2022',
 'December 2021',
 'November 2021',
 'October 2021',
 'September 2021']

In [9]:
current_month_idx = 8 # Aug

month_years = generate_month_year_list(2024, current_month_idx)
# month_years = ['March 2022', 'February 2022']
# month_years = ['February 2023', 'January 2023'] #, 'December 2022']

print(month_years)

# set up structure of df
description_col = 'Security Description'
security_col = 'Security No.'
holding_col = 'Holding'
value_col = 'Market Value'
percent_col = '% of Fund'
sheet_name = month_years[0]

jema_data = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name=sheet_name, skiprows=9, usecols=range(5))
jema_data = jema_data.dropna()
jema_data = jema_data.drop(columns=[holding_col, value_col, percent_col])

# get all unique entries by security_col
for month_year in month_years:
    sheet_name = month_year
    try:
        month_data = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name=sheet_name, skiprows=9, usecols=range(5))
        month_data = month_data.drop(columns=[holding_col, value_col, percent_col]).dropna()
        month_data[security_col] = month_data[security_col].replace('0H6400S', '2H6442S')

        # fix LUKOIL issue - drop the entries with the wrong security code, they'll be amended below
        month_data = month_data[month_data[security_col] != '0H6400S']
        
        # fix Rosneft issue - drop the entries with the wrong security code, they'll be amended below
        month_data = month_data[month_data[security_col] != '0H6367S']

        # drop subtotal row
        month_data = month_data[month_data[description_col] != 'Subtotal']

        jema_data = pd.concat([jema_data, month_data], ignore_index=True)
    except Exception as e:
        print(f"{sheet_name} failed")

print()

jema_data = jema_data.drop_duplicates([security_col])
jema_data = jema_data.reset_index(drop=True)

num_rows = len(jema_data)

jema_data_old = jema_data

for month_year in month_years:
    sheet_name = month_year
    try:
        month_data = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name=sheet_name, skiprows=9, usecols=range(5)).dropna()
        month_data = month_data.dropna()
        month_holding_col = f'{holding_col} {sheet_name}'
        month_value_col = f'{value_col} {sheet_name}'
        month_percent_col = f'{percent_col} {sheet_name}'

        month_data.rename(columns={holding_col: month_holding_col}, inplace=True)
        month_data.rename(columns={value_col:   month_value_col}, inplace=True)
        month_data.rename(columns={percent_col: month_percent_col}, inplace=True)
        month_data = month_data.dropna()

        # fix LUKOIL issue - amend wrong security code
        month_data[security_col] = month_data[security_col].replace('0H6400S', '2H6442S')

        # fix Rosneft issue  - amend wrong security code
        month_data[security_col] = month_data[security_col].replace('0H6367S', '2H7674S')

        # fix Novatek issue - amend wrong security code
        month_data.loc[(month_data[security_col] == '2H6464S') & (month_data[month_holding_col] ==  103572), security_col] = '0H5828S'

        # drop subtotal row
        month_data = month_data[month_data[description_col] != 'Subtotal']

        jema_data = jema_data.merge(month_data[[month_holding_col, security_col]], on=security_col, how='left')
        jema_data.fillna(0, inplace=True)

        if(len(jema_data_old) != len(jema_data)):
            duplicates = jema_data[jema_data[security_col].duplicated(keep=False)]

            print(f"\nRows with duplicate names {sheet_name}:")
            print(duplicates)
            break

        jema_data_old = jema_data

    except Exception as e:
        print(f"{sheet_name} failed")

jema_data.to_csv("jema_df.csv")
jema_data


['August 2024', 'July 2024', 'June 2024', 'May 2024', 'April 2024', 'March 2024', 'February 2024', 'January 2024', 'December 2023', 'November 2023', 'October 2023', 'September 2023', 'August 2023', 'July 2023', 'June 2023', 'May 2023', 'April 2023', 'March 2023', 'February 2023', 'January 2023', 'December 2022', 'November 2022', 'October 2022', 'September 2022', 'August 2022', 'July 2022', 'June 2022', 'May 2022', 'April 2022', 'March 2022', 'February 2022', 'January 2022', 'December 2021', 'November 2021', 'October 2021', 'September 2021']
February 2022 failed
January 2022 failed
December 2021 failed
November 2021 failed
October 2021 failed
September 2021 failed

February 2022 failed
January 2022 failed
December 2021 failed
November 2021 failed
October 2021 failed
September 2021 failed


,Security Description,Security No.,Holding August 2024,Holding July 2024,Holding June 2024,Holding May 2024,Holding April 2024,Holding March 2024,Holding February 2024,Holding January 2024,Holding December 2023,Holding November 2023,Holding October 2023,Holding September 2023,Holding August 2023,Holding July 2023,Holding June 2023,Holding May 2023,Holding April 2023,Holding March 2023,Holding February 2023,Holding January 2023,Holding December 2022,Holding November 2022,Holding October 2022,Holding September 2022,Holding August 2022,Holding July 2022,Holding June 2022,Holding May 2022,Holding April 2022,Holding March 2022
0,FIRSTRAND LTD COMMON STOCK ZAR 1,6606996,164763.00,164763.00,1.647630e+05,1.647630e+05,1.647630e+05,1.647630e+05,1.647630e+05,1.647630e+05,1.647630e+05,1.647630e+05,1.647630e+05,1.931570e+05,1.931570e+05,1.931570e+05,1.931570e+05,1.659400e+05,1.659400e+05,1.590930e+05,1.590930e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,AL RAJHI BANK COMMON STOCK SAR 10,B12LZH9,32585.00,32585.00,3.258500e+04,3.258500e+04,4.328300e+04,4.328300e+04,4.328300e+04,5.021400e+04,4.826100e+04,5.240200e+04,5.240200e+04,5.376300e+04,5.376300e+04,5.376300e+04,5.376300e+04,5.376300e+04,5.376300e+04,4.783500e+04,4.783500e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,JPM GBP LIQUIDITY LVNAV X (DIST.),5819115,514330.73,514330.73,5.143307e+05,5.143307e+05,5.143307e+05,5.143307e+05,5.143307e+05,5.007866e+05,1.000787e+06,1.000787e+06,1.000787e+06,1.000787e+06,1.000787e+06,7.856122e+05,1.000058e+06,1.000058e+06,1.000058e+06,1.005958e+06,2.029792e+06,1.700225e+07,1.700225e+07,1.700225e+07,1.698132e+07,1.698132e+07,1.706811e+07,1.707347e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,STANDARD BANK GROUP LTD COMMON STOCK ZAR 10,B030GJ7,47354.00,47354.00,4.735400e+04,4.735400e+04,4.735400e+04,4.735400e+04,4.735400e+04,4.735400e+04,4.735400e+04,4.735400e+04,4.735400e+04,5.467400e+04,4.501200e+04,4.501200e+04,4.501200e+04,4.501200e+04,4.501200e+04,4.501200e+04,4.501200e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,QATAR NATIONAL BANK QPSC COMMON STOCK QAR 1,6148197,142756.00,142756.00,1.173760e+05,1.173760e+05,1.173760e+05,1.173760e+05,1.312770e+05,1.312770e+05,1.312770e+05,1.312770e+05,1.312770e+05,1.312770e+05,1.459110e+05,1.459110e+05,1.459110e+05,1.459110e+05,1.459110e+05,1.459110e+05,1.459110e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
5,SAUDI ARABIAN OIL CO COMMON STOCK SAR,BJTM270,73566.00,73566.00,7.356600e+04,7.356600e+04,7.356600e+04,7.356600e+04,9.555000e+04,9.555000e+04,9.555000e+04,9.555000e+04,9.555000e+04,8.443200e+04,8.443200e+04,8.443200e+04,8.443200e+04,8.443200e+04,7.675700e+04,7.675700e+04,7.675700e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
6,EMAAR PROPERTIES PJSC COMMON STOCK AED 1,B01RM25,225483.00,225483.00,2.006730e+05,2.006730e+05,2.006730e+05,2.006730e+05,2.006730e+05,2.006730e+05,2.006730e+05,2.006730e+05,2.440400e+05,2.440400e+05,2.440400e+05,2.440400e+05,2.440400e+05,2.440400e+05,2.440400e+05,2.440400e+05,2.440400e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
7,THE SAUDI NATIONAL BANK,BSHYYN1,51456.00,51456.00,8.127100e+04,8.127100e+04,8.127100e+04,8.127100e+04,8.127100e+04,8.127100e+04,8.127100e+04,8.127100e+04,8.127100e+04,8.408900e+04,9.082500e+04,9.082500e+04,9.082500e+04,9.082500e+04,6.778600e+04,6.778600e+04,3.712200e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00

In [10]:
jema_data[jema_data[security_col] == '0H6400S'] # deleted

,Security Description,Security No.,Holding August 2024,Holding July 2024,Holding June 2024,Holding May 2024,Holding April 2024,Holding March 2024,Holding February 2024,Holding January 2024,Holding December 2023,Holding November 2023,Holding October 2023,Holding September 2023,Holding August 2023,Holding July 2023,Holding June 2023,Holding May 2023,Holding April 2023,Holding March 2023,Holding February 2023,Holding January 2023,Holding December 2022,Holding November 2022,Holding October 2022,Holding September 2022,Holding August 2022,Holding July 2022,Holding June 2022,Holding May 2022,Holding April 2022,Holding March 2022


In [11]:
jan23 = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name='January 2023',  skiprows=9, usecols=range(5)).dropna()
dec22 = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name='December 2022', skiprows=9, usecols=range(5)).dropna()
mar22 = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name='March 2022',    skiprows=9, usecols=range(5)).dropna()
apr22 = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name='April 2022',    skiprows=9, usecols=range(5)).dropna()
jun24 = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name='June 2024',     skiprows=9, usecols=range(5)).dropna()
apr22 = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name='April 2022',    skiprows=9, usecols=range(5)).dropna()
# joined = pd.merge(jan23, dec22, on=security_col, how='left')
# joined = mar22.merge(apr22, on=security_col, how='left')
# joined = jun24.merge(apr22, on=security_col, how='left')

sec = '2H6442S'

display(jan23[jan23[security_col] == sec])
display(dec22[dec22[security_col] == sec])
joined = pd.merge(jan23, dec22, on=security_col, how='left')
display(joined)

,Holding,Security Description,Market Value,% of Fund,Security No.
2,362649,LUKOIL PJSC COMMON STOCK RUB 0.025 RUB,216328.14,0.011464,2H6442S


,Holding,Security Description,Market Value,% of Fund,Security No.
2,362649,LUKOIL PJSC COMMON STOCK RUB 0.025 RUB,212831.74,0.01128,2H6442S


,Holding_x,Security Description_x,Market Value_x,% of Fund_x,Security No.,Holding_y,Security Description_y,Market Value_y,% of Fund_y
0,17002248.27,JPM GBP LIQUIDITY LVNAV X (DIST.),17002248.27,0.901035,5819115,17002248.27,JPM GBP LIQUIDITY LVNAV X (DIST.),17002248.27,0.901101
1,362649,LUKOIL PJSC COMMON STOCK RUB 0.025 RUB,216328.14,0.011464,2H6442S,362649,LUKOIL PJSC COMMON STOCK RUB 0.025 RUB,212831.74,0.011280
2,262047,LUKOIL PJSC COMMON STOCK RUB,132735.25,0.007034,0H5797S,262047,LUKOIL PJSC COMMON STOCK RUB,135847.01,0.007200
3,8860909,GAZPROM PJSC COMMON STOCK RUB,207280.93,0.010985,0H5796S,8860909,GAZPROM PJSC COMMON STOCK RUB,212140.29,0.011243
4,5365084,GAZPROM PJSC ADR,25337.18,0.001343,0H6364S,5365084,GAZPROM PJSC ADR,25931.17,0.001374
5,14455036,SBERBANK OF RUSSIA PJSC PREFERENCE RUB 3,186533.96,0.009885,0H5856S,14455036,SBERBANK OF RUSSIA PJSC PREFERENCE RUB 3,190906.94,0.010118
6,2355925,SBERBANK OF RUSSIA PJSC,30182.14,0.001600,0H5855S,2355925,SBERBANK OF RUSSIA PJSC,30889.71,0.001637
7,1268490,NOVATEK PJSC COMMON STOCK RUB 0.1,186325.77,0.009874,2H6464S,1268490,NOVATEK PJSC COMMON STOCK RUB 0.1,183314.28,0.009715
8,1268490,NOVATEK PJSC COMMON STOCK RUB 0.1,186325.77,0.009874,2H6464S,103572,NOVATEK PJSC COMMON STOCK RUB 0.1,13221.25,0.000701
9,103572,NOVATEK PJSC COMMON STOCK RUB 0.1,12918.40,0.000685,2H6464S,1268490,NOVATEK PJSC COMMON STOCK RUB 0.1,183314.28,0.009715
